# Limpieza de datos
Como punto inicial se importan los paquetes necesarios para el procesamiento de los datos

In [1]:
import numpy as np
import pandas as pd

Ahora se crea un data frame usando la función read_csv de pandas

In [2]:
df = pd.read_csv('Delitos.csv')
print(df[600:800])

      ID           HORA     FECHA    MES                     DELITO  \
600  601          19:40  23/01/16  ENERO                  EXTORSION   
601  602  02:30 A 05:30  24/01/16  ENERO           ROBO DE VEHICULO   
602  603           2:30  24/01/16  ENERO           ROBO DE VEHICULO   
603  604  02:30 A 03:30  24/01/16  ENERO           ROBO DE VEHICULO   
604  605  03:00 A 03:15  24/01/16  ENERO           ROBO DE VEHICULO   
605  606           0:00  24/01/16  ENERO           ROBO DE VEHICULO   
606  607          12:30  24/01/16  ENERO           ROBO DE VEHICULO   
607  608  12:30 A 16:30  24/01/16  ENERO           ROBO DE VEHICULO   
608  609  10:45 A 11:15  24/01/16  ENERO           ROBO DE VEHICULO   
609  610  11:30 A 11:40  24/01/16  ENERO           ROBO DE VEHICULO   
610  611          19:30  24/01/16  ENERO           ROBO DE VEHICULO   
611  612  21:30 A 22:00  24/01/16  ENERO           ROBO DE VEHICULO   
612  613           7:25  24/01/16  ENERO     ROBO A CUENTA HABIENTE   
613  6

Una vez cargados los datos en el data frame es posible acceder a los datos de cada columna usando el nombre correspondiente. Para ver las columnas contenidas en el dataframe basta con usar el atributo columns:

In [3]:
print(df.columns)

Index(['ID', 'HORA', 'FECHA', 'MES', 'DELITO', 'CALLE', 'No.', 'ENTRE_CALLE_1',
       'ENTRE_CALLE_2', 'COLONIA', 'SECTOR', 'CUADRANTE', 'TURNO', 'X', 'Y'],
      dtype='object')


De acuerdo a lo anterior, para imprimir la calle en la que fueron cometidos los primeros 10 delitos se puede utilizar ademas el método head().

In [4]:
print(df['CALLE'].head(10))

0      VALLE DE MEXICO
1        RANCHO GRANDE
2             LAURELES
3        LAGO GRAN OSO
4        LA CHAPARRITA
5               FAISAN
6      VALLE DE MEXICO
7        AV. PANTITLAN
8            CLAVELERO
9    VALLE DE IRAGUADI
Name: CALLE, dtype: object


De manera similar se puede utilizar el metodo tail() para imprimir los datos al final de la columna:

In [5]:
print(df['No.'].tail(20))

16405           NaN
16406           NaN
16407           NaN
16408           NaN
16409           NaN
16410           NaN
16411           NaN
16412           NaN
16413           NaN
16414           NaN
16415           NaN
16416           NaN
16417           154
16418    MZ.30 LT.8
16419           NaN
16420            48
16421           NaN
16422           NaN
16423           NaN
16424           NaN
Name: No., dtype: object


## Redondear valores

In [6]:
df = df.round({'X': 6, 'Y': 6})

## Rellenar campos vacíos

Como puede observarse varios de los registros de la columna "No." no tienen un valor asociado. Una de las labores principales de cualquier proceso de limpieza de datos es asegurarse que los mismos estén estandarizados, en consecuencia es necesario eliminar esta anomalia. Para tal fin, cada registro que no tenga un valor en la columna "No." serán llenado con la leyenda "Sin No.". Para realizar esta transformación basta con usar el metodo fillna() de una serie. Este método regresa una copia de la serie donde cada NaN es sustituido por el valor dado.

In [7]:
print(df['No.'].fillna('Sin No.').tail(20))

16405       Sin No.
16406       Sin No.
16407       Sin No.
16408       Sin No.
16409       Sin No.
16410       Sin No.
16411       Sin No.
16412       Sin No.
16413       Sin No.
16414       Sin No.
16415       Sin No.
16416       Sin No.
16417           154
16418    MZ.30 LT.8
16419       Sin No.
16420            48
16421       Sin No.
16422       Sin No.
16423       Sin No.
16424       Sin No.
Name: No., dtype: object


Ya que la columna 'No.' no es la única columna que presenta esta anomalia, es necesario realizar el proceso anterior para todas las columnas del dataframe.

In [8]:
for column in df.columns:
    df[column] = df[column].fillna('Sin ' + column)

El proceso anterior elimina las anomalias relacionadas con datos faltantes. Sin embargo, existen otros problemas que deben ser solucionados antes de que los datos puedan considerarse estandarizados. Por ejemplo, considere los datos que muestran la hora en la que fue cometido un delito.

In [9]:
print(df['HORA'].head(10))

0    22:30 A 04:00
1             6:40
2            13:30
3    03:00 A 05:00
4            14:00
5             0:05
6             4:00
7             4:58
8            18:13
9            19:40
Name: HORA, dtype: object


## Unificar el formato de las horas

Como se puede observar los datos tienen formatos diferentes. Existen registros que contienen una hora de inicio y una hora de fin. Con el objetivo de tener registro con entradas consistentes los datos de esta columna deben ser divididos. La columna 'HORA_INICIO' tendrá la hora en la que comenzó el delito, y la columna 'HORA_FIN' tendrá la hora en la que terminó el delito. Para aquellos registros que solo tengan una hora se tendra que la hora de inicio y la hora de fin será la misma.

In [10]:
df['HORA_INICIO'], df['HORA_FIN'] = df['HORA'].str.split(' A ', 1).str
df['HORA_FIN'] = np.where(pd.isna(df['HORA_FIN']), df['HORA_INICIO'], df['HORA_FIN'])

## Unificar el formato de las fechas

Se modificará el formato de las fechas de tal forma que todas ellas sean de la forma dd/mm/aaaa

In [11]:
import datetime as dt
#dfDate = df['FECHA']
#pd.to_datetime(dfDate, infer_datetime_format=True)
#df['FECHA'] = pd.to_datetime(dfDate, format = '%d/%m/%y')

#for row in bad_dates['FECHA'].str.split('-',3):
    #if len(row) is 2:
        #row.append('2017')
        
        #if len(row[0]) is 1:
            #row[0] = '0' + row[0]
            
        #print(row[0] + '/' + row[1] + '/' + row[2])
        
# First remove rows were month is given with a name
df['FECHA'] = df['FECHA'].str.replace('Jul', '07', regex=False)
df['FECHA'] = df['FECHA'].str.replace('ago', '08', regex=False)

# Now unify day, month, year separator
df['FECHA'] = df['FECHA'].str.replace('-', '/', regex=False)

# Add missing year
dates = df['FECHA'].str.extract(r'^((\d{2}|\d{1})/(\d{2}|\d{1})/(\d{4}|\d{2}))', expand=False)
#dates = df['FECHA'].str.extract(r'^((\d{2}|\d{1})/(\d{2}|\d{1})/(\d{4}))', expand=False)

df['FECHA'] = np.where(dates[0].isnull(), df['FECHA'] + '/2017', df['FECHA'])

dates = df['FECHA'].str.extract(r'(/(\d{2}))$', expand=False)
dates2 = df['FECHA'].str.extract(r'^((\d{2}|\d{1})/(\d{2}|\d{1})/)', expand=False)
#print(dates2)
df['FECHA'] = np.where(dates[0].notnull(), dates2[0] + '20' + dates[1], df['FECHA'])
print(df['FECHA'])

0          1/1/2016
1          1/1/2016
2          1/1/2016
3          1/1/2016
4          1/1/2016
5          1/1/2016
6          1/1/2016
7          1/1/2016
8          1/1/2016
9          1/1/2016
10         1/1/2016
11         1/1/2016
12         1/1/2016
13         1/1/2016
14         1/1/2016
15         1/1/2016
16         1/1/2016
17         1/1/2016
18         2/1/2016
19         2/1/2016
20         2/1/2016
21         2/1/2016
22         2/1/2016
23         2/1/2016
24         2/1/2016
25         2/1/2016
26         2/1/2016
27         2/1/2016
28         2/1/2016
29         2/1/2016
            ...    
16395    29/09/2017
16396    29/09/2017
16397    29/09/2017
16398    29/09/2017
16399     10/9/2017
16400     11/9/2017
16401     11/9/2017
16402    14/09/2017
16403    13/09/2017
16404    13/09/2017
16405    14/09/2017
16406    14/09/2017
16407    15/09/2017
16408    16/09/2017
16409    17/09/2017
16410    18/09/2017
16411    19/09/2017
16412    21/09/2017
16413    22/09/2017


## Eliminar espacios al inicio y final de la cadena de caracteres

In [12]:
for column in df.columns:
    df[column] = df[column].map(lambda x: str(x).strip())

## Cambiar el orden de las columnas

Para modificar el orden de las columnas agregadas recientemente y tenerlas de forma organizada, se puede hacer de la siguiente forma.

In [13]:
cols = df.columns.tolist()
cols

['ID',
 'HORA',
 'FECHA',
 'MES',
 'DELITO',
 'CALLE',
 'No.',
 'ENTRE_CALLE_1',
 'ENTRE_CALLE_2',
 'COLONIA',
 'SECTOR',
 'CUADRANTE',
 'TURNO',
 'X',
 'Y',
 'HORA_INICIO',
 'HORA_FIN']

In [14]:
cols = cols[:1] + cols[-2:] + cols[2:-2:]
cols

['ID',
 'HORA_INICIO',
 'HORA_FIN',
 'FECHA',
 'MES',
 'DELITO',
 'CALLE',
 'No.',
 'ENTRE_CALLE_1',
 'ENTRE_CALLE_2',
 'COLONIA',
 'SECTOR',
 'CUADRANTE',
 'TURNO',
 'X',
 'Y']

Se pone en este nuevo orden el dataframe.

In [15]:
df = df[cols]
print(df)

          ID HORA_INICIO HORA_FIN       FECHA         MES  \
0          1       22:30    04:00    1/1/2016       ENERO   
1          2        6:40     6:40    1/1/2016       ENERO   
2          3       13:30    13:30    1/1/2016       ENERO   
3          4       03:00    05:00    1/1/2016       ENERO   
4          5       14:00    14:00    1/1/2016       ENERO   
5          6        0:05     0:05    1/1/2016       ENERO   
6          7        4:00     4:00    1/1/2016       ENERO   
7          8        4:58     4:58    1/1/2016       ENERO   
8          9       18:13    18:13    1/1/2016       ENERO   
9         10       19:40    19:40    1/1/2016       ENERO   
10        11       21:10    21:10    1/1/2016       ENERO   
11        12        7:20     7:20    1/1/2016       ENERO   
12        13       20:09    20:09    1/1/2016       ENERO   
13        14       15:34    15:34    1/1/2016       ENERO   
14        15       18:53    18:53    1/1/2016       ENERO   
15        16        2:40

## Remover comas

En algunos casos el valor de un atributo puede contener una coma, esto no es deseable ya que se pueden producir erroes cuando se lee el archivo para ser procesado, en consecuencia es necesario eliminarlas.

In [16]:
df['No.'][1097]

'MZ. 130, LT. 4'

Para eliminarlas basta con utilizar el método ```replace()``` de una serie de la manera siguiente:

In [17]:
for column in df.columns:
    df[column] = df[column].str.replace(',', '.', regex=False)

In [18]:
df['No.'][1097]

'MZ. 130. LT. 4'

# Crear atributo 'coordenada'

Los atributos $X$ y $Y$ deben ser combinados en un solo atributo llamado COORDENADA, esta transformación es necesaria para utilizar una función de similitud basada en la coordenada en la que ocurrio un delito.

In [19]:
df['COORDENADA'] = df['X'] + ' ' + df['Y']
df = df.drop('X', 1)
df = df.drop('Y', 1)

In [20]:
df.to_csv('Delitos_cleaned.csv')